# PyVG: Data Science to predict Video Games sales
>Equipe: Alexis Terrasse, Henri-François Mole, Hsan Drissi, Stephane Lelievre
>
>Promo: DS_Oct21
---
## Exploratory data analysis 2 - MERGE des dataset scrappés
---
<img src="https://files.realpython.com/media/Merging-Joining-and-Concatenating-in-Pandas_Watermarked_1.e45698a508a4.jpg" width=450>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import seaborn as sns

In [2]:
def gen_dataset(r):
    # suppression des ventes nulles et des lignes avec des NaN
    #r = r.drop(r[r['Global_Sales']==0].index)
    r.dropna(inplace=True)
    
    # enregistrement
    r.to_csv("../data/vgsales_eda2.csv", sep=",", encoding='utf-8', index=False)
    
def add_game_key(df,col='Name'):
    """ Add normalized 'game_key' for col (default named 'Name')"""
    df['game_key'] = df['Name'].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8').str.replace('[^A-Za-z0-9]+', '-', regex=True).str.replace('^-+|-+$', '', regex=True).str.lower()  
    return df


In [3]:
# Renommage des plateformes, la reference est VGChartz
rename_platforms = {
    'Dreamcast': 'DC',
    'NDS':'DS',
    'NWii':'Wii',
    'Game Boy Advance': 'GBA',
    'GameCube': 'GC',
    'Nintendo 64' : 'N64',
    'PlayStation 2': 'PS2',
    'PlayStation 3': 'PS3',
    'PlayStation 4': 'PS4',
    'PlayStation 5': 'PS5',
    'PlayStation Vita': 'PSV',
    'PlayStation': 'PS',
    'Switch': 'NS',
    'Wii U' : 'WiiU',
    'Xbox 360' : 'X360',
    'Xbox One': 'XOne',
    'Xbox Series X': 'XS',
    'Xbox': 'XB',
    'ST':'Stadia',
}

platforms = ['2600','3DO','3DS','5200','7800','ACPC','AJ','AST','Aco','All','Amig','And','ApII','Arc','BBCM',
             'BRW','C128','C64','CD32','CDi','CV','DC','DS','DSi','DSiW','FMT','GB','GBA','GBC','GC','GEN','GG',
             'GIZ','Int','Linux','Lynx','MS','MSD','MSX','Mob','N64','NES','NG','NGage','NS','OR','OSX','Ouya',
             'PC','PCE','PCFX','PS','PS2','PS3','PS4','PS5','PSN','PSP','PSV','S32X','SAT','SCD','SNES','Series',
             'TG16','VB','VC','WS','WW','Wii','WiiU','WinP','X360','XB','XBL','XOne','XS','ZXS','iOS','iQue']

def check_platform(l):
    counter = 0
    print ('Verification des plateformes')
    for p in l:
        if p not in platforms:
            print (f' > {p} pas dans la liste des plateformes !')
            counter += 1
            
    print (f'Il y a {counter} plateforme(s) non conforme(s)')

### Source VGChartz

In [4]:
def read_vgchartz(file = "../data/vgchartz.csv"):

    name = file.split('/')[-1]
    
    #lecture du fichier
    df = pd.read_csv(file, sep=",")  
    
    # on elève les dates entre () dans le nom des jeux 
    df['Name'] = df['Name'].str.replace('\(\d+\)', '', regex=True) #<-- à voir si on garde ?? en fonction du gain sur la db

    # ajout de la clé de liaison
    df = add_game_key(df)
        
    # nettoyage
    df.dropna(subset=['Global_Sales'], inplace=True)
    #df["Global_Sales"].fillna('0m', inplace=True)
    df.replace(r'-for-nintendo.*$','',regex=True, inplace=True)
    
    # conversion de la colonne Global Salas en float
    df["Global_Sales"] = df["Global_Sales"].apply(lambda x: x[:-1]).astype('float64')
    
    # remove duplicates
    df.drop_duplicates(['game_key','Platform'],keep='first',inplace=True)
    
    print (f'{name}: {df.shape} - unique keys {len(df.game_key.unique())}')
    
    return df
    
vgchartz = read_vgchartz()
vgchartz.head(3)

vgchartz.csv: (20209, 5) - unique keys 13898


,Name,Platform,Genre,Global_Sales,game_key
0,Grand Theft Auto: San Andreas,PS2,Action,20.81,grand-theft-auto-san-andreas
1,Grand Theft Auto V,PS3,Action,20.32,grand-theft-auto-v
2,Grand Theft Auto V,PS4,Action,19.39,grand-theft-auto-v


### Source Metacritic

#### Corrections meta trouvées au fur et à mesure

In [5]:
def meta_correction(df_in):
    df_in.loc[df_in['Publisher'].str.contains('10tons'),'Publisher'] = '10tons'
    df_in.loc[df_in['Publisher'].str.contains('11 bit'),'Publisher'] = '11 bit'
    df_in.loc[df_in['Publisher'].str.contains(r'^1c'),'Publisher'] = '1c entertainment'
    df_in.loc[df_in['Publisher'].str.contains('2k games'),'Publisher'] = '2k games'

    df_in.loc[df_in['Publisher'].str.contains('ycjy'),'Publisher'] = 'ycjy'
    df_in.loc[df_in['Publisher'].str.contains('20th century fox'),'Publisher'] = '20th century fox'
    df_in.loc[df_in['Publisher'].str.contains('3rd eye studios'),'Publisher'] = '3rd eye studios'
    df_in.loc[df_in['Publisher'].str.contains('young horses'),'Publisher'] = 'young horses'

    df_in.loc[df_in['Publisher'].str.contains('505 games'),'Publisher'] = '505 games'
    df_in.loc[df_in['Publisher'].str.contains('amc'),'Publisher'] = 'amc games'
    df_in.loc[df_in['Publisher'].str.contains('ankama'),'Publisher'] = 'ankama'
    df_in.loc[df_in['Publisher'].str.contains('atari'),'Publisher'] = 'atari'

    df_in.loc[df_in['Publisher'].str.contains('badland'),'Publisher'] = 'badland'
    df_in.loc[df_in['Publisher'].str.contains('bandai'),'Publisher'] = 'bandai'
    df_in.loc[df_in['Publisher'].str.contains('bitbox'),'Publisher'] = 'bitbox'
    df_in.loc[df_in['Publisher'].str.contains('bethesda'),'Publisher'] = 'bethesda games'

    df_in.loc[df_in['Publisher'].str.contains('black pants'),'Publisher'] = 'black pants'
    df_in.loc[df_in['Publisher'].str.contains('blizzard'),'Publisher'] = 'blizzard'
    df_in.loc[df_in['Publisher'].str.contains('buena vista'),'Publisher'] = 'buena vista games'
    df_in.loc[df_in['Publisher'].str.contains('milestone'),'Publisher'] = 'milestone srl'

    df_in.loc[df_in['Publisher'].str.contains('activision'),'Publisher'] = 'activision'
    df_in.loc[df_in['Publisher'].str.contains('sega'),'Publisher'] = 'sega'
    df_in.loc[df_in['Publisher'].str.contains('thq'),'Publisher'] = 'thq'
    df_in.loc[df_in['Publisher'].str.contains('sony'),'Publisher'] = 'sony interactive entertainment'
    df_in.loc[df_in['Publisher'].str.contains('koei'),'Publisher'] = 'koei games'
    df_in.loc[df_in['Publisher'].str.contains('agetec'),'Publisher'] = 'agetec inc'
    df_in.loc[df_in['Publisher'].str.contains('valve'),'Publisher'] = 'valve'
    df_in.loc[df_in['Publisher'].str.contains('alawar'),'Publisher'] = 'alawar'
    
def copy_score(r):
    if pd.isnull(r['Score_user']):
        return r['Score_pro']
    elif r['Score_user']==0:
        return r['Score_pro']
    else:
        return r['Score_user']

In [6]:
def read_metacritic(file = "../data/metacritic.csv"):

    name = file.split('/')[-1]
    
    #lecture du fichier
    df = pd.read_csv(file, sep=",")  

    # on elève les dates entre () dans le nom des jeux 
    df['Name'] = df['Name'].str.replace('\(\d+\)', '', regex=True)
    
    # ajout de la clé de liaison
    df = add_game_key(df)
    
    # renommage des features
    df.rename(columns={"Meta_Critic_Score": "Score_pro",
                       'Meta_User_Score':'Score_user',
                       "N_Critic_Reviews": "N_pro",
                       'N_User_Reviews':'N_user'}, inplace=True)

    # renommage des plateformes
    df['Platform'].replace(rename_platforms, inplace=True)

    # ajout de la date et date decomposée
    df[['month_str', 'day', 'year']] = df['Release_date'].str.extract(r'(\w+) +(\d+), +(\d+)')
    df['month_str'] = df['month_str'].fillna(df['month_str'].mode()[0])
    df['month'] = df['month_str'].apply(lambda x: datetime.strptime(x, "%b").month if (type(x) == str) else x)
    df['release_date']= pd.to_datetime(df[['year','month', 'day']])
    df['quarter'] = df['release_date'].dt.quarter
    df = df.drop(columns=['Release_date','month_str'])

    # normalization à minima des publishers --> servira par la suite pour savoir s'ils sont importants ou non
    feat = 'Publisher'
    df[feat] = df[feat].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
    df[feat] = df[feat].str.replace('[^A-Za-z0-9,]+', ' ', regex=True).str.replace('^ +| +$', '', regex=True).str.lower()
    df[feat] = df[feat].str.replace(' +', ' ', regex=True).str.replace(' ?, ?',',', regex=True)
    df[feat] = df[feat].str.replace(',inc', '', regex=True)
    df[feat].fillna('',inplace=True)
    
    # correction de dataframe
    meta_correction(df)
    
    # mise à l'echelle des score pro
    df['Score_pro'] = df['Score_pro'] / 10
    
    # et MAJ score user manquant à partir des scores pro --> on s'autorise cette aproximation
    df['Score_user'] = df.apply(copy_score,axis=1)

    # remove duplicates and remaning N/A
    df.drop_duplicates(['game_key','Platform'],keep='first',inplace=True)
    df.dropna(inplace=True)
    
    print (f'{name}: {df.shape} - unique keys {len(df.game_key.unique())}')
    
    # verification que les plateformes sont biens reconnues
    check_platform(df['Platform'].unique())
    
    return df

meta = read_metacritic()
meta.head(3)
#19203

metacritic.csv: (19203, 15) - unique keys 12379
Verification des plateformes
 > Stadia pas dans la liste des plateformes !
Il y a 1 plateforme(s) non conforme(s)


,Name,Platform,Genre,Score_pro,Score_user,Publisher,Developer,N_pro,N_user,game_key,day,year,month,release_date,quarter
0,Full Throttle,PC,"Adventure, General",8.6,8.6,lucasarts,LucasArts,8,196,full-throttle,30,1995,4,1995-04-30,2.0
1,Sid Meier's Civilization II,PC,"Strategy, Turn-Based, Historic, General, Histo...",9.4,8.7,microprose,MPS Labs,7,484,sid-meier-s-civilization-ii,29,1996,2,1996-02-29,1.0
2,Quake,PC,"Action, Shooter, Shooter, First-Person, Sci-Fi...",9.4,8.8,id software,id Software,9,934,quake,22,1996,6,1996-06-22,2.0


### Source IGDB

In [7]:
# Franchises connues de wikipedia
liste_des_franchises = ['Zup','1080° Snowboarding','1942','3D Ultra Minigolf','3-D Ultra Pinball','7th Dragon','A Boy and His Blob','Ace Attorney','Ace Combat','ActRaiser','Adventure Island','Adventures of Lolo','Aero Fighters','Aero the Acro-Bat','After Burner','Age of Empires','Age of Wonders','Airforce Delta','Aleste','Alex Kidd','Alien Breed','Alien Syndrome','Alone in the Dark','Alpine Racer','Altered Beast','Alundra','American McGee s Alice','America s Army','Amnesia','Amped','Angry Birds','Animal Crossing','Anno','Anomaly','Another Century s Episode','Another Code','Ape Escape','Arc the Lad','Arkanoid','ARMA','Armored Core','Army Men','Army of Two','Art Academy','Ar Tonelico','Asheron s Call','Asphalt','Assassin s Creed','Assault Heroes','Asteroids','Astro','Atelier','ATV Offroad Fury','Audiosurf','Avadon','Babylonian Castle Saga','Backyard Sports','Baldur s Gate','Bangai-O','Banjo-Kazooie','Baraduke','Bard s Tale','Baseball Stars','Bases Loaded','Batman','Battle Arena Toshinden','Battlefield','Battle Isle','Battlestations','Battletoads','Battlezone','Bayonetta','Beat Hazard','Beatmania','Bendy','Bejeweled','Big Brain Academy','Bionic Commando','BioShock','Bit.Trip','The Black Mirror','Black & White','Blaster Master','BlazBlue','Blazing Angels','Blinx','Blitz: The League','Blood','Blood Bowl','BloodRayne','Bloody Roar','Blue Dragon','Bobby Carrot','Boktai','Boku no Natsuyasumi','Bomberman','Bomb Jack','Bonk','Boom Blox','Border Break','Borderlands','Bosconian','Boulder Dash','BoxBoy!','Brain Age','Bravely','Breakout','Breath of Fire','Broken Sword','Brothers in Arms','Bubble Bobble','Bubsy','Budget Cuts','Burgertime','Burnout','Bushido Blade','Bus Simulator','Buster Bros.','Bust a Groove','Buzz!','Cabela s','Call of Duty','Call of Juarez','Candy Crush','Cannon Fodder','Capcom Vs. SNK','Carmageddon','Car Mechanic Simulator','Carmen Sandiego','Carnival Games','Castle Shikigami','Castlevania','Centipede','Championship Manager','Chaos Rings','Chase H.Q.','Chessmaster','Chibi-Robo!','Chivalry','Choplifter','Chrono','City Connection','Civilization','ClayFighter','Clicker Heroes','Clock Tower','Clockwork Knight','Clubhouse Games','Colin McRae Rally','Colony Wars','Columns','Combat Mission','Command & Conquer','Commandos','Company of Heroes','Condemned','The Conduit','Conker','Contra','Cooking Mama','Cool Boarders','Corpse Party','Cotton','Counter-Strike','Crackdown','Crash Bandicoot','Crazy Castle','Crazy Taxi','Creatures','The Crew','Crimson Skies','Croc','Cruis n','Crusader Kings','Crush Pinball','Crysis','The Culling','Custom Robo','Cut the Rope','Cyber Sled','Dance Central','Dance Dance Revolution','Danganronpa','Darius','Dark Cloud','Dark Seed','Darksiders','Darkstalkers','Daytona USA','DC Comics','de Blob','Dead Frontier','Dead Island','Deadly Premonition','Dead or Alive','Dead Rising','Dead Space','Dead to Rights','Deathsmiles','DeathSpank','Deer Hunter','Defense Grid','Def Jam','Defender','Delta Force','Densha de Go!','Descent','Desperados','Destroy All Humans!','Deus Ex','Devil Children','Devil May Cry','Diablo','Dies irae','Dig Dug','Dino Crisis','Disaster Report','Disgaea','Dishonored','Disney Infinity','Dizzy','Donkey Kong','DonPachi','Doom','Dota','Double Dragon','Dragon Age','Dragon Ball','Dragon Buster','Dragon Force','Dragon Quest','Dragon Slayer','Dragon Spirit','Dragon s Lair','Drakengard','Drawn to Life','Dream Chronicles','Driver','Duke Nukem','Dungeon Defenders','Dungeon Explorer','Dungeon Keeper','Dungeon Siege','Dying Light','Dynasty Warriors','Earth Defense Force','Earthworm Jim','Ecco the Dolphin','El Dorado Gate','The Elder Scrolls','Elevator Action','Empire Earth','Eternal Champions','Etrian Odyssey','Europa Universalis','EverQuest','Everybody s Golf','Evil Genius','The Evil Within','eXceed','Excite','Exerion','Exit','EyeToy','F1 Circus','F-Zero','Fable','Fallout','Famicom Grand Prix','Famicom Tantei Club','Family Game Night','Family Party','Family Stadium','The Fancy Pants Adventures','Fantasy Zone','Far Cry','Far East of Eden','Farming Simulator','Fatal Frame','Fatal Fury','Fat Princess','F.E.A.R.','Fieldrunners','FIFA','Fighting Vipers','Final Fantasy','Final Lap','Fire Emblem','Five Nights at Freddy s','FlatOut','Flowers','Football Manager','Forza','Fossil Fighters','Frequency','Frogger','Front Mission','Frostpunk','G-Police','Gabriel Knight','Gal Gun','Galaxian','Galaxy Force','Game Tengoku','Ganbare Goemon','Gauntlet','Gears of War','Gekisha Boy','Geneforge','Genpei Tōma Den','Geometry Wars','The Getaway','Gex','Ghosts  n Goblins','Giana Sisters','Giga Wing','Glory of Heracles','God Eater','God of War','Golden Axe','Golden Sun','The Golf Club','Golly! Ghost!','Gothic','Gradius','Gran Turismo','Grand Theft Auto','Grandia','Groove Coaster','Ground Control','Growlanser','Guacamelee!','Guardian Heroes','Guild','Guild Wars','Guilty Gear','Guitar Hero','Gunbird','Gundam','Gunpey','Gunslinger Stratos','Gunstar Heroes','Gunvolt','.hack','Half-Life','Halo','Hammerin  Harry','Hang-On','Hard Drivin ','Harry Potter','Harvest Moon','Hatsune Miku: Project DIVA','Hat Trick Hero','Hearts of Iron','Hebereke','Heiankyo Alien','Herzog','Hexic','Hidden & Dangerous','Hitman','Homeworld','Hotline Miami','The House of the Dead','Hyperdimension Neptunia','Hydlide','Hydro Thunder','Icewind Dale','The Idolmaster','Ikari Warriors','Illusion','Image Fight','Imagine','Inazuma Eleven','Infamous','Infinity','Infinity Blade','International Superstar Soccer','Iron Soldier','Itadaki Street','Jagged Alliance','Jak and Daxter','Jake Hunter','James Bond','Jazz Jackrabbit','J.B. Harold','Jetpac','Jet Moto','Jet Set Radio','Joe & Mac','Joe Danger','The Journeyman Project','Juiced','Jumping Flash!','Just Cause','Just Dance','Kane & Lynch','Katamari','Keyboardmania','Kid Icarus','Kid Niki','KiKi KaiKai','Killer Instinct','Killing Floor','Killzone','Kinect Sports','Kingdom Hearts','The King of Fighters','King s Quest','Kirby','Kirby Fighters','Klonoa','Knights of the Old Temple','Kotoba no Puzzle','Kunio-kun','Kururin','Kyle Hyde','Lands of Lore','Langrisser','The Last Blade','Last Ninja','The Last of Us','League of Legends','Left 4 Dead','Legacy of Kain','Legend of Legaia','The Legendary Starfy','The Legend of Heroes','The Legend of Kage','The Legend of Zelda','Hyrule Warriors','Lego','Leisure Suit Larry','Lemmings','Lethal Enforcers','Life Is Strange','Lineage','Lips','LittleBigPlanet','Little Nightmares','Lode Runner','The Lord of the Rings','Lost Kingdoms','Lost Planet','The Lost Vikings','Lotus','LovePlus','Lucky s Tale','Lufia','Luigi','Lumines','Lunar','Madden NFL','Mafia','Magical Drop','Mana','Manhunt','Maniac Mansion','Mappy','Mario','Dr. Mario','Mario Kart','Mario Party','Mario sports','Mario RPG','Mario vs. Donkey Kong','Super Mario','Marvel','Marvel vs. Capcom','Mass Effect','Master of Orion','Math Blaster','Max Payne','MechAssault','MechWarrior','Medal of Honor','MediEvil','Mega Man','Megami Tensei','Mercenaries','Metal Gear','Metal Max','Metal Slug','Metro','Metroid','Microsoft Combat Flight Simulator','Microsoft Flight Simulator','Midnight Club','Midtown Madness','Might and Magic','Milon s Secret Castle','Minecraft','Mirror s Edge','MLB 2K','MLB: The Show','Momoko 120%','Momotaro Densetsu','Momotaro Dentetsu','Monaco GP','Monkey Island','Monster Hunter','Monster Rancher','Monster Truck Madness','Monument Valley','Mortal Kombat','Mother','Motocross Madness','MotoGP','Moto Racer','MotorStorm','Mount & Blade','Mr. Do!','Mr. Driller','MX vs. ATV','Myst','Mystery Dungeon','Myth','Namco Anthology','Namco Museum','Nancy Drew','Naruto: Ultimate Ninja','NASCAR','Navy Field','NBA 2K','NBA Live','NBA Jam','Nectaris / Military Madness','Need for Speed','Nekopara','NES Remix','Neutopia','Neverwinter Nights','NFL 2K','NHL (EA Sports) ','NHL 2K','Nidhogg','Ni no Kuni','Nights','Ninja Gaiden','Ninja JaJaMaru-kun','Nintendogs','Nobunaga s Ambition','No More Heroes','Numan Athletics','Oddworld','Ogre','Ōkami','One Piece','One Must Fall','Onimusha','Operation Wolf','Orcs Must Die!','The Oregon Trail','Osu! Tatakae! Ouendan','Otogi','Otomedius','Outlast','Out Run','Overcooked','Pac-Man','Pac-Man World','Panzer Dragoon','PaRappa the Rapper','Parasite Eve','Parkan','Parodius','Patapon','Pathologic','Payday','Peggle','Pengo','Penumbra','Perfect Dark','Persona','Petz','PGA Tour','Phantasy Star','Pikmin','Pillars of Eternity','Pilotwings','Pinball FX','Pirate Ship Higemaru','Pitfall!','PixelJunk','PlanetSide','Plants vs. Zombies','Point Blank','Pokémon','Pokémon Pinball','Pokémon Play It!','Pokémon Puzzle','PokéPark','Pokémon Snap','Pokémon Stadium','Pole Position','Pong','Populous','Portal','Postal','Power Pros','Power Stone','Prince of Persia','Princess Maker','Pro Evolution Soccer','Professor Layton','Project Gotham Racing','Project X Zone','Prototype','Punch-Out!!','Pushmo','Putt-Putt','Puyo Puyo','Puzzle Bobble','Puzzle & Dragons','Puzzle League','Q*bert','Qix','Quake','Quest for Glory','R-Type','Raiden','Railroad Tycoon','Rakugaki Ōkoku','Rampage','Rally-X','Rastan','Ratchet & Clank','RayForce','Rayman','Raving Rabbids','R.C. Pro-Am','Red Dead','Red Faction','Red Steel','Resident Evil','Resistance','Retro Game Challenge','Return Fire','Rhythm Heaven','Rick Dangerous','Ridge Racer','Risen','Risk of Rain','Road Fighter','Road Rash','Robopon','Robot','Robotron: 2084','Rock Band','Rod Land','Rogue Legacy','RollerCoaster Tycoon','Rolling Thunder','Romance of the Three Kingdoms','RPG Maker','Rugby League','Rumble Roses','Rune Factory','Runescape','Rush','Rush n Attack','Rygar','Sabreman','SaGa','Saints Row','Sakura Wars','Salamander','Sam & Max','Samurai Shodown','Samurai Warriors','Sanctum','Scene It?','Schoolgirl Strikers','Science Adventure','Scramble','Scribblenauts','Sea Battle','Seaman','Sega Ages','Sega Bass Fishing','Sega Rally','Sengoku Basara','Serious Sam','The Settlers','Shadowgate','Shadow Man','Shadow of the Beast','Shadow Warrior','Shank','Shantae','Shenmue','Shining','Shinobi','Shōnen Jump','Shoot Away','Silent Hill','Silent Scope','Silpheed','SimCity','Simple','The Sims','Sin and Punishment','SingStar','Skate','Skylanders','Sly Cooper','Snake Rattle  n  Roll','Sniper Elite','Snowboard Kids','SOCOM','Soldier of Fortune','Sonic Blast Man','Sonic the Hedgehog','Soulcalibur','Souls','Space Channel 5','Space Harrier','Space Invaders','Space Quest','Speedball','Spider-Man','Splashdown','Splatoon','Splatterhouse',' Splosion Man','SpongeBob SquarePants','Spy Hunter','Spyro','SSX','S.T.A.L.K.E.R.','Star Control','Star Force','Star Fox','Star Luster','Star Ocean','Star Raiders','Star Wars','StarCraft','Star Soldier','StarTropics','State of Decay','SteamWorld','Steel Battalion','Steel Division','Steel Gunner','Story of Seasons','Street Fighter','Streets of Rage','Strider','Strikers 1945','Stronghold','Subnautica','Suikoden','Summon Night','Super Mega Baseball','Super Monkey Ball','Super Robot Wars','Super Smash Bros.','Supreme Commander','Surgeon Simulator','Sutte Hakkun','Swordquest','Syberia','Syphon Filter','System Shock','Taiko no Tatsujin','Taito Memories','Tak','Tales','Tank Battalion','Team Fortress','Tecmo Bowl','Teenage Mutant Ninja Turtles','Tekken','Tempest','Tenchu','Terra Cresta','Test Drive','Tetris','Thief','Thunder Ceptor','Thunder Cross','Thunder Force','Tiger Heli','Time Crisis','Time Pilot','TimeSplitters','Titanfall','Tobal','TOCA','ToeJam & Earl','Tokimeki Memorial','Tom Clancy s','Tom Clancy s The Division','Tom Clancy s Ghost Recon','Tom Clancy s H.A.W.X','Tom Clancy s Rainbow Six','Tom Clancy s Splinter Cell','Tomb Raider','Tomodachi Collection','Tony Hawk s','Torchlight','Total Annihilation','Total War','Touch! Generations','Touhou Project','Toukiden','Track & Field','Train Simulator','Transformers','Trauma Center','Trials','Tribes','Trine','Tropico','Truck Simulator','True Crime','Truxton','Turok','Turrican','TwinBee','Twisted Metal','Two Worlds','Uncharted','Unravel','Unreal','Uridium','Valis','Valkyria Chronicles','Valkyrie','Valkyrie Profile','Vampire: The Masquerade','Vandal Hearts','Vanguard','Vectorman','Vib-Ribbon','Viewtiful Joe','Vigilante 8','Violence Fight','Virtua Cop','Virtua Fighter','Virtua Striker','Virtua Tennis','Virtual On','Viva Piñata','V-Rally','Wagan Land','The Walking Dead','Wangan Midnight','Warcraft','Warhammer 40,000','Warhammer Fantasy','Wario','WarioWare','Wario Land','Warlords','Wars','Wasteland','Watch Dogs','Wave Race','White Knight Chronicles','Wii','Wild Arms','Wing Commander','Winning Run','Wipeout','The Witcher','Wizardry','Wizards & Warriors','Wolf Fang','Wolfenstein','Wonder Boy','Wonder Momo','Wonder Project','World Heroes','World Stadium','Worms','Wrecking Crew','WWE 2K','Xanadu','X-COM','Xeno','Xevious','Yakuza','Yie Ar Kung-Fu','Yo-kai Watch','Yooka-Laylee','Yoshi','You Don t Know Jack','Ys','Yu-Gi-Oh!','Zanac','Zaxxon','Zero Escape','Zill O ll','Zombie Tycoon','Zone of the Enders','Zool','Zoombinis','Zoo Tycoon','Zork','Zuma','Zumba Fitness']
len(liste_des_franchises)

902

In [8]:
# Etape d'ajout des franchises wikipedia
def find_franchise(r):
    res = ""
    
    if not pd.isnull(r['Franchise']):     # s'il y a une franchise déjà identifiée, on la prend
        res = r['Franchise']        
    else:
        for i in liste_des_franchises:    # sinon, on regarde si on en trouve une parmi la liste wikipedia
            if i in r['Name']:            # dans le titre du jeux
                if res == "":
                    res=i
                else:
                    res=res+','+i
            
        if res == "":                     # si on en a toujours pas trouvé on prend le nom du jeu
            res=r['Name'].split(':')[0]   # et si on trouve un ':', seulement ce qu'il y a avant le ":"
        else:
            size = 0
            lres = res.split(',')         # si on en a une, on prend le nom le plus grand (proba plus faible)
            for i in lres:
                if len(i) > size:
                    size=len(i)
                    res = i
    return res

In [9]:
def read_igdb(file = "../data/igdb_franchises.csv"):

    name = file.split('/')[-1]
    
    # lecture du fichier
    df = pd.read_csv(file, sep=",")  
    
    # ajout de la clé de liaison --> la notre est plus consistente avec les titres
    df = add_game_key(df)
    
    # Consolidation des franchises
    df['Franchise_wikipedia'] = df.apply(find_franchise,axis=1 )
    
    # on retire ce qui n'est plus utile
    #df.drop(['Name', 'Platform', 'Franchise', 'Release date'], axis=1,inplace=True)
    #df.drop_duplicates(subset='game_key',inplace=True)
    
    print (f'{name}: {df.shape} - unique keys {len(df.game_key.unique())}')
    
    return df[['game_key','Franchise_wikipedia']]
    
igdb = read_igdb()
igdb.head(3)
# 28930 --> 28118 

igdb_franchises.csv: (28930, 7) - unique keys 28118


,game_key,Franchise_wikipedia
0,0-day-attack-on-earth,0 Day Attack on Earth
1,0-ji-no-kane-to-cinderella-halloween-wedding,0 Ji no Kane to Cinderella ~Halloween Wedding~
2,007-legends,James Bond


### NLP - META PRO

In [11]:
def already_processed_nlp_meta(file = "../data/meta_prorev_withsentiments.csv"):
    name = file.split('/')[-1]
    
    #lecture du fichier
    df = pd.read_csv(file, sep=",")  
    
    # ajout de la clé de liaison --> la notre est plus consistente avec les titres
    df = add_game_key(df)

    # renommage des plateformes
    df['Platform'].replace(rename_platforms, inplace=True)
    
    df = df.groupby(['game_key','Platform'],as_index=False).mean()

    print (f'{name}: {df.shape} - unique keys {len(df.game_key.unique())}')
    check_platform(df['Platform'].unique())
    
    return df[['game_key','Platform','compound']]
    
nlp_meta = already_processed_nlp_meta()
nlp_meta.head(5)

meta_prorev_withsentiments.csv: (19185, 7) - unique keys 12398
Verification des plateformes
 > Stadia pas dans la liste des plateformes !
Il y a 1 plateforme(s) non conforme(s)


,game_key,Platform,compound
0,0-day-attack-on-earth,X360,0.031400
1,007-legends,PS3,-0.133464
2,007-legends,X360,-0.093835
3,007-nightfire,PC,-0.094536
4,007-quantum-of-solace,DS,0.229530


### wikipedia - single/multi player

In [12]:
def read_wikiplayer(file = "../data/vgsales_wikipedia.csv"):

    name = file.split('/')[-1]
    
    # lecture du fichier
    df = pd.read_csv(file, sep=",")  
    
    # ajout de la clé de liaison --> la notre est plus consistente avec les titres
    df = add_game_key(df)
    
    df.drop_duplicates(subset='game_key',inplace=True)
    print (f'{name}: {df.shape} - unique keys {len(df.game_key.unique())}')
    
    return df[['game_key','multiplayer','singleplayer']]

wiki_players = read_wikiplayer()
# vgsales_wikipedia.csv: (5438, 6) - unique keys 5431

vgsales_wikipedia.csv: (5431, 6) - unique keys 5431


### Merge final

In [13]:
vm = pd.merge(vgchartz,meta,on=['game_key','Platform'],how='inner',suffixes=('','_meta'))
print ('VM shape:', vm.shape)

vmi = pd.merge(vm,igdb,on=['game_key'],how='inner',suffixes=('','_igdb'))
print ('VMI shape:',vmi.shape)

vmiv = pd.merge(vmi,nlp_meta,on=['game_key','Platform'],how='inner',suffixes=('','_vader'))
print ('VMIV shape:',vmiv.shape)

vmivw = pd.merge(vmiv,wiki_players,on=['game_key'],how='left',suffixes=('','_wiki'))
print ('VMIVW shape:',vmivw.shape)

#VM shape: (9131, 18)
#VMI shape: (9055, 19)
#VMIV shape: (8938, 20)
#VMIVW shape: (8938, 22)

VM shape: (9131, 18)
VMI shape: (9055, 19)
VMIV shape: (8938, 20)
VMIVW shape: (8938, 22)


In [14]:
# on rectifie les NaN
vmivw['multiplayer'].fillna(0,inplace=True)
vmivw['singleplayer'].fillna(1,inplace=True)
vmivw['compound'].fillna(vmivw['compound'].mean(),inplace=True)

# et petite verif
vmivw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8938 entries, 0 to 8937
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Name                 8938 non-null   object        
 1   Platform             8938 non-null   object        
 2   Genre                8938 non-null   object        
 3   Global_Sales         8938 non-null   float64       
 4   game_key             8938 non-null   object        
 5   Name_meta            8938 non-null   object        
 6   Genre_meta           8938 non-null   object        
 7   Score_pro            8938 non-null   float64       
 8   Score_user           8938 non-null   float64       
 9   Publisher            8938 non-null   object        
 10  Developer            8938 non-null   object        
 11  N_pro                8938 non-null   int64         
 12  N_user               8938 non-null   int64         
 13  day                  8938 non-nul

In [15]:
# ecriture du dataset mergé
gen_dataset(vmivw)